# Exploring the Allen Brain Observatory Database
In this notebook, we'll explore data from the allen Brain Observatory database. See their [documentation](http://help.brain-map.org/display/observatory/Documentation) for more details about the mice, experimental setup, data processing, and analysis.

For an example notebook from the Allen Institute that demonstrates how to work with the data, see [here](http://alleninstitute.github.io/AllenSDK/_static/examples/nb/brain_observatory.html).

### Two recent preprints that have made use of this data:
1. Christensen and Pillow (2017). [Running reduces firing but improves coding in rodent higher-order visual cortex](https://www.biorxiv.org/content/early/2017/11/04/214007)
2. Esfahany et al. (2017). [Organization of Neural Population Code in Mouse Visual System](https://www.biorxiv.org/content/early/2017/12/04/220558)

### Also see
* [This helpful blog post](https://kachio.github.io/blog/2016/12/10/Decoding-Identity-of-Natural-Images) on wrangling the Brain Observatory datasets and natural imaging decoding from Onyekachi Odoemene in Anne Churchland's lab. Some of the code for importing is modified from this post, as well as the AllenSDk example.


In [1]:
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import allensdk.brain_observatory.stimulus_info as stim_info
from allensdk.brain_observatory.natural_scenes import NaturalScenes
import numpy as np
import h5py
import os
# This class uses a 'manifest' to keep track of downloaded data and metadata.  
# All downloaded files will be stored relative to the directory holding the manifest
# file.  If 'manifest_file' is a relative path (as it is below), it will be 
# saved relative to your working directory.  It can also be an absolute path.
boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

In [2]:
import pprint
targeted_structures = boc.get_all_targeted_structures()
print("all targeted structures: " + str(targeted_structures))

cre_lines = boc.get_all_cre_lines()
print("all cre lines: " + str(cre_lines))

all targeted structures: [u'VISal', u'VISam', u'VISl', u'VISp', u'VISpm', u'VISrl']
all cre lines: [u'Cux2-CreERT2', u'Emx1-IRES-Cre', u'Nr5a1-Cre', u'Rbp4-Cre_KL100', u'Rorb-IRES2-Cre', u'Scnn1a-Tg3-Cre']


In [9]:
experiments = boc.get_ophys_experiments(stimuli=['natural_scenes'],
                                        targeted_structures =['VISl', 'VISp'],
                                        session_types=[stim_info.THREE_SESSION_B],
                                       cre_lines=['Cux2-CreERT2'])
len(experiments)

29

In [5]:
import pandas as pd

# The following line takes a while to run the first time
cells = boc.get_cell_specimens()
cells = pd.DataFrame.from_records(cells)
print("total cells: %d" % len(cells))

total cells: 37091


In [7]:
cells.head()

,all_stim,area,cell_specimen_id,donor_full_genotype,dsi_dg,experiment_container_id,failed_experiment_container,g_dsi_dg,g_osi_dg,g_osi_sg,...,specimen_id,tfdi_dg,time_to_peak_ns,time_to_peak_sg,tld1_id,tld1_name,tld2_id,tld2_name,tlr1_id,tlr1_name
0,False,VISpm,517394847,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,0.852081,511498500,False,0.468690,0.275754,NaN,...,503292439,0.348821,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
1,False,VISpm,517394850,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,0.730537,0.342070,NaN,...,503292439,0.298012,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
2,False,VISpm,517394854,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,0.692137,511498500,False,0.319382,0.215510,NaN,...,503292439,0.380637,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
3,False,VISpm,517394858,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,NaN,NaN,0.76123,...,503292439,NaN,0.4655,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)
4,False,VISpm,517394862,Cux2-CreERT2/Cux2-CreERT2;Camk2a-tTA/wt;Ai93(T...,NaN,511498500,False,0.391861,0.742900,NaN,...,503292439,0.330543,NaN,NaN,177839004,Cux2-CreERT2,177837320,Camk2a-tTA,265943423,Ai93(TITL-GCaMP6f)


In [17]:
ec_ids = [ ec['experiment_container_id'] for ec in experiments ]
natural_scences_cells = cells[cells['experiment_container_id'].isin(ec_ids)]
natural_scences_cells.shape

(9677, 60)

In [19]:
natural_scences_cells.columns

Index([u'all_stim', u'area', u'cell_specimen_id', u'donor_full_genotype',
       u'dsi_dg', u'experiment_container_id', u'failed_experiment_container',
       u'g_dsi_dg', u'g_osi_dg', u'g_osi_sg', u'image_sel_ns',
       u'imaging_depth', u'osi_dg', u'osi_sg', u'p_dg', u'p_ns',
       u'p_run_mod_dg', u'p_run_mod_ns', u'p_run_mod_sg', u'p_sg',
       u'peak_dff_dg', u'peak_dff_ns', u'peak_dff_sg', u'pref_dir_dg',
       u'pref_image_ns', u'pref_ori_sg', u'pref_phase_sg', u'pref_sf_sg',
       u'pref_tf_dg', u'reliability_dg', u'reliability_nm1_a',
       u'reliability_nm1_b', u'reliability_nm1_c', u'reliability_nm2',
       u'reliability_nm3', u'reliability_ns', u'reliability_sg',
       u'rf_area_off_lsn', u'rf_area_on_lsn', u'rf_center_off_x_lsn',
       u'rf_center_off_y_lsn', u'rf_center_on_x_lsn', u'rf_center_on_y_lsn',
       u'rf_chi2_lsn', u'rf_distance_lsn', u'rf_overlap_index_lsn',
       u'run_mod_dg', u'run_mod_ns', u'run_mod_sg', u'sfdi_sg', u'specimen_id',
       u'tfdi_

In [ ]:
# this takes a long time to run and downloads a ton of data
for i,exp in enumerate(experiments):
    print(i)
    boc.get_ophys_experiment_data(exp['id']) 

0


2017-12-11 15:26:26,024 allensdk.api.api.retrieve_file_over_http INFO     Downloading URL: http://api.brain-map.org/api/v2/well_known_file_download/514430242
